In [478]:
import pandas as pd
import synonyms
import jieba
import jieba.posseg as pseg

In [2]:
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

height has been deprecated.



In [697]:
corpus_df = pd.read_csv('/Users/olap/Desktop/corpus.csv').reset_index(drop = True)
corpus_2_list = corpus_df[corpus_df['rate'] == 2]['token'].tolist()
corpus_3_list = corpus_df[corpus_df['rate'] == 3]['token'].tolist()

In [278]:
def title_token_rater(tokens):
    
    tokens_rate = list(map(lambda x:3 if x in corpus_3_list else(2 if x in corpus_2_list else 0), tokens))

    return tokens_rate

In [279]:
def title_syn_rater(tokens):
    
    syn_3_cnt = 0
    syn_2_cnt = 0
    syn_cnt_list = []
    
    for token in tokens:
        syn = synonyms.nearby(token)[0]
        syn_rates = list(map(lambda x:3 if x in corpus_3_list else(2 if x in corpus_2_list else 0), syn))

        syn_cnt = [2 if syn_rates.count(3) < syn_rates.count(2) else(0 if syn_rates.count(3) == 0 and syn_rates.count(2) == 0 else 3)]
        syn_cnt_list += syn_cnt
        
    return syn_cnt_list

In [353]:
def title_processor(title):
    
    tokens = list(set(list(jieba.cut_for_search(title))))
#     print(tokens)
    token_cnt_list = np.array(title_token_rater(tokens))
    syn_cnt_list = np.array(title_syn_rater(tokens))
    
    token_non_zero = np.where(token_cnt_list > 0)
    syn_cnt_list = np.delete(syn_cnt_list, token_non_zero)
            
    token_cnt_list = token_cnt_list.tolist()
    syn_cnt_list = syn_cnt_list.tolist()
    
    syn_3_cnt = syn_cnt_list.count(3)
    syn_2_cnt = syn_cnt_list.count(2)
    token_3_cnt = token_cnt_list.count(3)
    token_2_cnt = token_cnt_list.count(2)
    
    return [token_2_cnt, token_3_cnt, syn_2_cnt, syn_3_cnt]

In [82]:
def processor_testing(title, rate):
    tokenized_title = list(set(list(jieba.cut_for_search(title))))
    for token in tokenized_title:
    #     print (token)
        token_syn = [token] + synonyms.nearby(token)[0]
        print (token_syn)
        
        if rate == 2:
            print (list(map(lambda x:x in corpus_2_list, token_syn)))
        if rate == 3:
            print (list(map(lambda x:x in corpus_3_list, token_syn)))

In [113]:
def token_testing(token):
    
    print (list(jieba.cut_for_search(token)))
    print (synonyms.nearby(token))

In [79]:
training_title_list = open('/Users/olap/Desktop/training_title_list.txt', 'r').read().split('\n')[:100]
title_feature_list = list(map(title_processor, training_title_list))

ture_label = open('/Users/olap/Desktop/training_set.txt', 'r').read().split('\n')
true_label_list = list(map(lambda x:x.split(' ')[-1], ture_label))
true_label_list = [1 if label=='0' else label for label in true_label_list]
true_label_list = [0 if label=='100' else label for label in true_label_list][:-1]

label_list = list(map(lambda x:x.split(',')[-1], title_rate))

In [602]:
title = pd.DataFrame(training_title_list)
feature = pd.DataFrame(title_feature_list)
true_label = pd.DataFrame(true_label_list)

noobs_df = pd.concat([title, feature, true_label], axis= 1)
noobs_df.columns = ['title', 'token_2', 'token_3', 'syn_2', 'syn_3', 'true_label']

In [83]:
import numpy as np
X = np.array(noobs_df.iloc[:,1:5])
y = np.array(noobs_df.iloc[:,-1])

In [549]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=0)

for train_index, test_index in sss.split(X, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

1

In [558]:
from sklearn.ensemble import RandomForestClassifier
# model = RandomForestClassifier()
from sklearn import linear_model
model = linear_model.LogisticRegression()

In [559]:
model.fit(X_train, y_train)
model.score(X_train, y_train)

0.90000000000000002

In [560]:
model.score(X_test, y_test)

0.93333333333333335

In [561]:
model_predict = pd.DataFrame(model.predict(X_test))
model_predict.columns = ['model_pred']
model_proba = pd.DataFrame(model.predict_proba(X_test))
model_proba.columns = ['proba_0', 'proba_1']

index = test_index.tolist()
noobs_df_test = noobs_df.iloc[index].loc[:, noobs_df.columns != 'label'].reset_index(drop = True)
noobs_df_test = pd.concat([noobs_df_test, model_predict, model_proba], axis= 1)

In [562]:
# keyword based model mistake cases
noobs_df_test[((noobs_df_test['token_3'] == 0) & (noobs_df_test['true_label'] == 1))
             | (noobs_df_test['token_3'] > 0) & (noobs_df_test['true_label'] == 0)]

,Title,token_2,token_3,syn_2,syn_3,true_label,model_pred,proba_0,proba_1
13,网络惊现售卖原味丝袜二手内裤 交易背后不堪入目_新闻频道_中华网,3,0,2,0,1,1,0.032174,0.967826
26,外围女真实生活揭秘：一夜遍体鳞伤换来30万_食神的神_食神的神的和讯博客,2,0,2,0,1,1,0.150053,0.849947
28,大尺度无底线直播 – 【人人分享-人人网】,3,0,0,0,1,1,0.039918,0.960082
29,"舆情分析：兰陵一副校长致初中女生""怀孕"" - 案例库 - 华声舆情 - 华声在线",1,0,1,0,1,0,0.511835,0.488165


In [569]:
noobs_df_test

,Title,token_2,token_3,syn_2,syn_3,true_label,model_pred,proba_0,proba_1
0,“速度与激情”版烧烤炉横空出世-奇闻007,1,0,0,0,0,0,0.539714,0.460286
1,大麻会对生活造成危害吗？ - 知乎,0,1,0,0,1,0,0.513932,0.486068
2,护肤不贵| 高性价二次清洁爽肤水盘点,0,0,1,0,0,0,0.847747,0.152253
3,广场大全景再现 国庆花坛扮靓京城_网易新闻,0,0,0,0,0,0,0.861628,0.138372
4,2017年秋季白领事业信心指数下降，跳槽比例高,0,0,0,0,0,0,0.861628,0.138372
5,十八子作雅致七件套刀团购价格-国美团购,0,0,0,0,0,0,0.861628,0.138372
6,"7年,怎么从一个三流学校的高职生到自主创业的CEO - 挨踢生活 - 51CTO技术论坛_中...",0,0,1,0,0,0,0.847747,0.152253
7,2012消费电子博览会 NEC科技打造“智慧城市”－青岛新闻网,0,0,1,0,0,0,0.847747,0.152253
8,九只梅花鹿亮相北京故宫博物院(组图)_新闻频道_中华网,0,0,1,0,0,0,0.847747,0.152253
9,—在线播放—优酷网，视频高清在线观看,0,0,2,0,0,0,0.832743,0.167257


In [686]:
yewai_df = pd.read_csv('/Users/olap/Desktop/yw_title.csv').drop('Unnamed: 0', axis  = 1)
yewai_df = yewai_df[yewai_df.true_label.notnull()].reset_index(drop = True)

In [574]:
rf = RandomForestClassifier()
lr = linear_model.LogisticRegression()

In [698]:
def model_perfermance(data, test_size, model, diff):
    
    title_list = data['title'].tolist()
    X = np.array(list(map(title_processor, title_list)))
    y = np.array(data['true_label'].tolist())
    
    title = pd.DataFrame(title_list)
    feature = pd.DataFrame(X)
    true_label = pd.DataFrame(y)

    df = pd.concat([title, feature, true_label], axis= 1)
    df.columns = ['title', 'token_2', 'token_3', 'syn_2', 'syn_3', 'true_label']
    
    sss = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=0)
    
    for train_index, test_index in sss.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
    
    model.fit(X_train, y_train)
    
    print('traning acc', model.score(X_train, y_train))
    print('test acc', model.score(X_test, y_test))

    
    model_predict = pd.DataFrame(model.predict(X_test))
    model_predict.columns = ['pred_label']
    model_proba = pd.DataFrame(model.predict_proba(X_test))
    model_proba.columns = ['proba_0', 'proba_1']
    
    index = test_index.tolist()
    test_df = df.iloc[index].reset_index(drop = True)
    test_df = pd.concat([test_df, model_predict, model_proba], axis= 1)
    
        
    rule_base_mis = len(test_df[((test_df['token_3'] == 0) & (test_df['true_label'] == 1))
             | (test_df['token_3'] > 0) & (test_df['true_label'] == 0)])
    
    print('rule base acc', 1-rule_base_mis/len(test_df))
    
    mis_df = test_df[test_df['true_label'] != test_df['pred_label']]
    manual_rating_df = test_df[abs(test_df['proba_0']-test_df['proba_1']) < diff] 
    manual_rating_cnt = len(manual_rating_df)
    manual_improve_cnt = len(test_df[(test_df['true_label'] != test_df['pred_label'])
                            & (abs(test_df['proba_0']-test_df['proba_1']) < diff)])
    
    
    print('test_count', len(test_df))
    print('manually rate title', manual_rating_cnt)
    print('can improve', manual_improve_cnt)
    
    if manual_rating_cnt != 0:
        print('manyally eff', manual_improve_cnt/manual_rating_cnt)
    
    print('model + manual acc', model.score(X_test, y_test) + manual_improve_cnt/len(test_df))
    

    return mis_df

In [699]:
model_perfermance(yewai_df, .3, lr, .3)

traning acc 0.808743169399
test acc 0.898734177215
rule base acc 0.810126582278481
test_count 79
manually rate title 15
can improve 5
manyally eff 0.3333333333333333
model + manual acc 0.962025316456


,title,token_2,token_3,syn_2,syn_3,true_label,pred_label,proba_0,proba_1
10,·找越南农村女人做老婆 卖给朋友被判刑,1,0,2,0,1.0,0.0,0.552413,0.447587
19,·尼格买提娇妻美照曝光 新疆美女气质迷人,3,0,0,0,0.0,1.0,0.182474,0.817526
36,·女子嫌乳房太大缩胸失败告医院!,1,0,1,0,1.0,0.0,0.609659,0.390341
55,·曝刘翔葛天分居:刘翔葛天婚后分居,1,0,1,0,1.0,0.0,0.609659,0.390341
59,·盘点世界最“重口味”菜肴!,1,0,0,0,1.0,0.0,0.664036,0.335964
65,·胎盘饮料、母牛液体 盘点全球最诡异的饮料！,1,0,2,0,1.0,0.0,0.552413,0.447587
67,·环卫工被打破眼球 劝路人别乱扔垃圾,0,0,0,0,1.0,0.0,0.854686,0.145314
70,·女子遭弃尸公寓水塔 居民喝怪味水一年,1,0,2,1,1.0,0.0,0.567241,0.432759


In [685]:
test[(test['token_2'] > 0) & (test['true_label'] == 0)]

,title,token_2,token_3,syn_2,syn_3,true_label
0,·酒店的5个奇怪服务，你受得了吗？,1,0,1,0,0.0
2,·搞笑动图之美女系列，真的很养眼！,1,0,0,0,0.0
3,·范冰冰赵薇 明星母亲超牛家底背景曝光(组图),1,0,0,0,0.0
7,·夏日颜值爆表的极品美女都是这样穿的，诱惑到底！,2,0,1,0,0.0
9,·赵丽颖陈晓分手 赵丽颖陈晓分手是真吗,1,0,0,0,0.0
15,·那些被皇帝爱到骨子里的绝色佳人(组图),1,0,2,0,0.0
16,·看看女神明星的卸妆照，瞬间自信心爆棚！,1,0,0,0,0.0
21,·女教师微博秀性感身躯爆红 腹肌完美(组图),1,0,1,0,0.0
23,·为啥男人都喜欢20多岁的美女？,1,0,0,0,0.0
26,·中国美女都在哪些城市 结果出乎人们意料！,1,0,0,0,0.0


In [700]:
processor_testing('陈赫张子萱热吻 公寓内做饭时贴身缠绵', 3)

['缠绵', '离别', '深情', '痴情', '一段情', '柔情', '爱恋', '相逢', '抒发', '热恋', '真情']
[True, False, False, False, False, False, False, False, False, False, False]
['热吻']
[False]
['时']
[False]
['萱', '娟', '俊', '姬', '芷', '淑', '馨', '张文', '玲', '琛', '贞']
[False, False, False, False, False, False, False, False, False, False, False]
['陈赫', '白百何', '黄圣依', '姚晨', '刘亦菲', '杨洋', '林心如', '杨幂', '赵薇', '倪妮', '李双江']
[False, False, False, False, False, False, False, False, False, False, False]
['内']
[False]
[' ']
[False]
['做饭', '吃饭', '洗衣服', '洗澡', '饭菜', '煮饭', '打扫', '干活', '取暖', '家务', '家里']
[False, False, False, False, False, False, False, False, False, False, False]
['公寓', '公寓楼', '旅馆', '房子', '住宅', '别墅', '楼房', '住所', '豪宅', '酒店', '洋房']
[False, False, False, False, False, False, False, False, False, False, False]
['贴身', '保镖', '保镳', '装扮', '礼服', '西装', '年轻貌美', '打扮', '易容', '军装', '侍从']
[False, False, False, False, False, False, False, False, False, False, False]
['张子']
[False]


In [570]:
token_testing('大麻会对生活造成危害吗？ - 知乎')

['大麻', '会', '对', '生活', '造成', '危害', '造成危害', '吗', '？', ' ', '-', ' ', '知乎']
[[], []]
